In [ ]:
region = 'eu'

In [ ]:
%%sql
/* SQL: Import players from database */
SELECT * FROM "Players"

In [ ]:
%%sql
/* SQL: Import posts from database */
SELECT * FROM "Posts"

In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Posts: The Flairs column is actually a bitfield, so we need to convert it to a list of flairs :
# Neutral=0
# Perfomance: Good=1 Bad=2
# Teamplay: Good=4 Bad=8
# Courtessy: Good=16 Bad=32

# We will use the following function to convert the flairs bitfield into a series of nullable booleans
class PostFlairs:
    Neutral = 0x00
    PerformanceGood = 0x01
    PerformanceBad = 0x02
    TeamplayGood = 0x04
    TeamplayBad = 0x08
    CourtesyGood = 0x10
    CourtesyBad = 0x20

def flairs_to_maskvalue(flairs, neg_mask, pos_mask):
    flairs_bit = bin(flairs)
        
    if flairs is None or 0: return 'neutral'
    elif flairs & pos_mask != 0: return 'positive'
    elif flairs & neg_mask != 0: return 'negative'
    else: return 'neutral'

def flairs_to_list(flairs):
    return [
        flairs_to_maskvalue(flairs, PostFlairs.PerformanceBad, PostFlairs.PerformanceGood), # Performance
        flairs_to_maskvalue(flairs, PostFlairs.TeamplayBad, PostFlairs.TeamplayGood), # Teamplay
        flairs_to_maskvalue(flairs, PostFlairs.CourtesyBad, PostFlairs.CourtesyGood) # Courtesy
    ]

In [ ]:
# Process flairs and add each property to the dataframe
df_posts_flairs = df_posts.copy()
[df_posts_flairs['Performance'], df_posts_flairs['Teamplay'], df_posts_flairs['Courtesy']] = zip(*df_posts_flairs['Flairs'].apply(flairs_to_list))

In [ ]:
df_posts_flairs

In [ ]:
# Define the post type as a median of the flairs : Negative gives -1, Neutral gives 0, Positive gives 1.
# The post type is determined as greater, lesser, or equal to 0, which gives us respectively the following types : Positive, Negative, Neutral
def flairs_to_type(metrics):
    balance = 0
    
    for metric in metrics:
        if metric == 'positive' : balance += 1
        elif metric == 'negative': balance -= 1
    
    return 'positive' if balance > 0 \
        else 'negative' if balance < 0 \
        else 'neutral'

In [ ]:
# Process flairs and add the post type to the dataframe
df_posts_flairs['Type'] = df_posts_flairs[['Performance', 'Teamplay', 'Courtesy']].apply(flairs_to_type, axis=1)

In [ ]:
df_posts_flairs

In [ ]:
# Export the dataframe to a JSON file
df_posts_flairs.to_json(f'data/{region}_posts.json', indent=1, orient='records')